<a href="https://colab.research.google.com/github/mhuckvale/pals0039/blob/master/Exercise_4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![PALS0039 Logo](https://www.phon.ucl.ac.uk/courses/pals0039/images/pals0039logo.png)](https://www.phon.ucl.ac.uk/courses/pals0039/)

#Exercise 4.3

In this exercise we implement a DNN for predicting age of the speaker from speech.

The data comes from the [Accents of the British Isles](http://www.thespeechark.com/abi-2-page.html) corpus.

(a) Set up for learning. Runn the code and add comments.

In [0]:
# 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

---
(b) Load in the data as two spreadsheets. Run the code and add comments.

In [0]:
# 
def LoadCSV(filename,labelname,featcolstart):
  # 
  df=pd.read_csv(filename,sep=',')
  # 
  gender=pd.Categorical(df['GENDER'])
  # 
  feats=np.append(np.array(df.iloc[:,featcolstart:]),np.reshape(gender.codes,(len(df),1)),axis=1)
  # 
  labels=df[labelname].to_numpy()
  #
  p = np.random.permutation(feats.shape[0])
  return labels[p],feats[p]

# 
ytrain,Xtrain=LoadCSV("https://www.phon.ucl.ac.uk/courses/pals0039/data/age-train.csv","AGE",4);
ytest,Xtest=LoadCSV("https://www.phon.ucl.ac.uk/courses/pals0039/data/age-test.csv","AGE",4);

# 
ntrain=Xtrain.shape[0]
ntest=Xtest.shape[0]
isize=Xtrain.shape[1]
osize=1
print("ntrain",ntrain,"ntest",ntest,"isize",isize,"osize",osize)

---
(c) Normalise the features. Run the code and add comments

In [0]:
# 
mn=np.mean(Xtrain,axis=0)
sd=np.std(Xtrain,axis=0)
sd[(sd<0.001)]=1

# 
Xtrain=(Xtrain-mn)/sd;
Xtest=(Xtest-mn)/sd;

# 
ytrain=ytrain/100
ytest=ytest/100

---
(d) Build a model. Run the code and add comments.

In [0]:
# 
model = Sequential()
# 
model.add(Dense(64,activation='tanh',input_shape=(isize,)))
# 
model.add(Dropout(0.5))
# 
model.add(Dense(32,activation='tanh'))
# 
model.add(Dense(osize, activation='linear'))
# 
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
print(model.summary())

(e) Fit model. Run code and add comments.

In [0]:
# 
hist=model.fit(Xtrain,ytrain, epochs=100, batch_size=32, validation_data=(Xtest,ytest))

(f) Evaluate the model on test data. Run the code and add comments

In [0]:
# 
mseloss,maeloss=model.evaluate(Xtest,ytest)
print("mseloss",mseloss,"maeloss",maeloss*100,"years")

(g) Plot predictions. Run the code and add comments

In [0]:
# 
ypred=model.predict(Xtest)

#
x=ytest*100
y=ypred*100

# 
colours=['red','blue']
cols=list(colours[int(i)] for i in Xtest[:,isize-1])
# 
plt.scatter(x=x,y=y,c=cols,s=20,alpha=0.5);
# 
poly = np.polyfit(x, y, 1)
x=np.array((np.min(x),np.max(x)))
y=poly[1]+poly[0]*x
plt.plot(x,y,':r')
plt.xlabel("Actual Age (yr)");
plt.ylabel("Predicted Age (yr)");
plt.grid()
plt.title("Age prediction");
plt.show()
